# Test Process for Image Analysis

In [ ]:
import json
import numpy as np
import pandas as pd
#import pHash
from urllib.parse import urlparse
import requests
import time
import shutil
import sys

pd.set_option('max_colwidth',500)
pd.set_option('max_rows', 300)

###Import JSON to Data Frame

In [ ]:
#filename = '/Users/cbopp/Data/Nepal/nepal-earthquake-4-27-2248-head-1k.json'
#filename = '/Users/cbopp/Data/Nepal/nepal-earthquake-4-27-2248-head-25.json'
filename = '/Users/cbopp/Data/Sandy_Image/2012_Hurricane_Sandy-2k.jsonl'
l = []

for line in open(filename):
    l.append(json.loads(line))

df = pd.DataFrame(l)
df['created_at'] = pd.to_datetime(df['created_at'], format="%a %b %d %H:%M:%S +0000 %Y")
df['created_at_epoch'] = df['created_at'].astype('int64')
df['text'] = df['text'].apply(lambda s: s.replace('\n', ' '))
df['text'] = df['text'].apply(lambda s: s.replace('\r', ' '))
df['text'] = df['text'].apply(lambda s: s.replace('\t', ' '))

In [ ]:
retweeted_status_data = {'tweet_id_str': [], 'rt': [], 'original_tweet_created_at': [], 'original_tweet_id_str': [], 'original_user_created_at': [], 'original_user_id_str': [], 'original_user_default_avatar': [], 'original_user_followers_count': [], 'original_user_friends_count': [], 'original_user_listed_count': [], 'original_user_name': [], 'original_user_tweet_count': [], 'original_user_verified': [], 'original_user_description': []}

for tweet in l:  
    try:
        tweet['retweeted_status']
        rt = True
    except:
        rt = False
        
    if rt:
        retweeted_status_data['tweet_id_str'].append(tweet['id_str'])
        retweeted_status_data['rt'].append(rt)
        retweeted_status_data['original_tweet_created_at'].append(tweet['retweeted_status']['created_at'])
        retweeted_status_data['original_tweet_id_str'].append(tweet['retweeted_status']['id_str'])
        retweeted_status_data['original_user_created_at'].append(tweet['retweeted_status']['user']['created_at'])
        retweeted_status_data['original_user_id_str'].append(tweet['retweeted_status']['user']['id_str'])
        retweeted_status_data['original_user_default_avatar'].append(tweet['retweeted_status']['user']['default_profile_image'])
        retweeted_status_data['original_user_followers_count'].append(tweet['retweeted_status']['user']['followers_count'])
        retweeted_status_data['original_user_friends_count'].append(tweet['retweeted_status']['user']['friends_count'])
        retweeted_status_data['original_user_listed_count'].append(tweet['retweeted_status']['user']['listed_count'])
        retweeted_status_data['original_user_name'].append(tweet['retweeted_status']['user']['name'])
        retweeted_status_data['original_user_tweet_count'].append(tweet['retweeted_status']['user']['statuses_count'])
        retweeted_status_data['original_user_verified'].append(tweet['retweeted_status']['user']['verified'])
        try:
            retweeted_status_data['original_user_description'].append(tweet['retweeted_status']['user']['description'])
        except:
            retweeted_status_data['original_user_description'].append(np.NaN)
    else:
        retweeted_status_data['tweet_id_str'].append(tweet['id_str'])
        retweeted_status_data['rt'].append(rt)
        retweeted_status_data['original_tweet_created_at'].append(np.NaN)
        retweeted_status_data['original_tweet_id_str'].append(np.NaN)
        retweeted_status_data['original_user_created_at'].append(np.NaN)
        retweeted_status_data['original_user_id_str'].append(np.NaN)
        retweeted_status_data['original_user_default_avatar'].append(np.NaN)
        retweeted_status_data['original_user_followers_count'].append(np.NaN)
        retweeted_status_data['original_user_friends_count'].append(np.NaN)
        retweeted_status_data['original_user_listed_count'].append(np.NaN)
        retweeted_status_data['original_user_name'].append(np.NaN)
        retweeted_status_data['original_user_tweet_count'].append(np.NaN)
        retweeted_status_data['original_user_verified'].append(np.NaN)
        retweeted_status_data['original_user_description'].append(np.NaN)

retweeted_status = pd.DataFrame(retweeted_status_data, columns=['tweet_id_str', 'rt', 'original_tweet_created_at', 'original_tweet_id_str', 'original_user_created_at', 'original_user_id_str', 'original_user_default_avatar', 'original_user_followers_count', 'original_user_friends_count', 'original_user_listed_count', 'original_user_name', 'original_user_tweet_count', 'original_user_verified', 'original_user_description'])

retweeted_status['original_user_description'].replace('', np.NaN, inplace=True)

retweeted_status['original_tweet_created_at'] = pd.to_datetime(retweeted_status['original_tweet_created_at'], format="%a %b %d %H:%M:%S +0000 %Y")
retweeted_status['original_user_created_at'] = pd.to_datetime(retweeted_status['original_user_created_at'], format="%a %b %d %H:%M:%S +0000 %Y")
retweeted_status['original_tweet_created_at_epoch'] = retweeted_status['original_tweet_created_at'].astype('int64')
retweeted_status['original_user_created_at_epoch'] = retweeted_status['original_user_created_at'].astype('int64')

In [ ]:
user_data = {'tweet_id_str': [], 'user_created_at': [], 'user_id_str': [], 'user_default_avatar': [], 'user_followers_count': [], 'user_friends_count': [], 'user_listed_count': [], 'user_name': [], 'user_tweet_count': [], 'user_verified': [], 'user_description': []}

for tweet in l:
    user_data['tweet_id_str'].append(tweet['id_str'])
    user_data['user_created_at'].append(tweet['user']['created_at'])
    user_data['user_id_str'].append(tweet['user']['id_str'])
    user_data['user_default_avatar'].append(tweet['user']['default_profile_image'])
    user_data['user_followers_count'].append(tweet['user']['followers_count'])
    user_data['user_friends_count'].append(tweet['user']['friends_count'])
    user_data['user_listed_count'].append(tweet['user']['listed_count'])
    user_data['user_name'].append(tweet['user']['name'])
    user_data['user_tweet_count'].append(tweet['user']['statuses_count'])
    user_data['user_verified'].append(tweet['user']['verified'])
    try:
        user_data['user_description'].append(tweet['user']['description'])
    except:
        user_data['user_description'].append(np.NaN)
        
user = pd.DataFrame(user_data, columns=['tweet_id_str', 'user_created_at', 'user_id_str', 'user_default_avatar', 'user_followers_count', 'user_friends_count', 'user_listed_count', 'user_name', 'user_tweet_count', 'user_verified', 'user_description'])

user['user_description'].replace('', np.NaN, inplace=True)
user['user_created_at'] = pd.to_datetime(user['user_created_at'], format="%a %b %d %H:%M:%S +0000 %Y")
user['user_created_at_epoch'] = user['user_created_at'].astype('int64')

###Identify Tweets that Contain Extended Entities

In [ ]:
def embedded_images(e):
    l = []
    try:
        for o in e['media']:
            url = o['media_url']
            l.append(url)
        return l
    except:
        pass

df['e_media'] = df['entities'].apply(embedded_images)
df['contains_image_by_e'] = df.e_media.apply(lambda o: type(o) == list)

In [ ]:
e_media_data = {'tweet_id_str': [], 'media_url': [], 'image_id_str': [], 'image_original_tweet_id_str': []}

for tweet in l:
    for entity in tweet['entities']:
        if entity == 'media':
            for image in tweet['entities'][entity]:
                e_media_data['tweet_id_str'].append(tweet['id_str'])
                e_media_data['media_url'].append(image['media_url'])
                e_media_data['image_id_str'].append(image['id_str'])
                try:
                    e_media_data['image_original_tweet_id_str'].append(image['source_status_id_str'])
                except:
                    e_media_data['image_original_tweet_id_str'].append(np.NaN)

e_media = pd.DataFrame(e_media_data, columns=['tweet_id_str', 'image_original_tweet_id_str', 'image_id_str', 'media_url'])

In [ ]:
#For newer Twitter data sets that contain the extended_entities object
def embedded_images(ee):
    l = []
    for o in ee['media']:
        url = o['media_url']
        l.append(url)
    return l

df['ee_media'] = df[df.extended_entities.notnull()]['extended_entities'].apply(embedded_images)
df['contains_image_by_ee'] = df.ee_media.apply(lambda o: type(o) == list)

###Unwind URLs

In [ ]:
df['urls'] = df.entities.apply(lambda u: u['urls'])
df['url_count'] = df['urls'].apply(lambda l: len(l))

In [ ]:
link_shorteners = ['bit.ly','goo.gl','fb.me','is.gd'] #TODO: Add more

def resolve_urls(ent):
    l = []
    session = requests.Session()
    session.mount("http://", requests.adapters.HTTPAdapter(max_retries=2))
    session.mount("https://", requests.adapters.HTTPAdapter(max_retries=2))
    
    for o in ent['urls']:
        url = urlparse(o['expanded_url'])
        if url.netloc in link_shorteners:
            try:
                l.append(session.get(url=url.geturl(),timeout=6).url)
                #TODO: If the connection fails, the URL doesn't get appended. Fix to append anyway.
                #TODO: Should this be a dict instead of original_url: expanded_url?
            except requests.exceptions.RequestException as e:
                print(e)
        else:
            l.append(url.geturl())
    return l

df['expanded_urls'] = df.entities.apply(resolve_urls)

In [ ]:
to_search = 'blog.meconfess.com'
#to_search = 'www.myopenforum.com'
#to_search = 'findmejob.org'
#to_search = 'www.laopinion.com'
#to_search = 'visibli.com'
#to_search = 'buzzfeed'
df[df.expanded_urls.str[0].str.contains(to_search, case=False, na=False)]

df[df.expanded_url

In [ ]:
df[df.expanded_urls.str[1].str.contains(to_search, case=False, na=False)]

###Identify Images by HTTP Content Type

In [ ]:
def content_type(urls):
    l = []
    for url in urls:
        try:
            sys.stdout.write('\r')
            sys.stdout.write(url)
            sys.stdout.flush()
            r = requests.get(url, timeout=0.1)
            l.append(r.headers['Content-Type'].split(sep=';')[0])
        except:
            pass #TODO: Make this less horrible.
    return l

df['content_type'] = df.expanded_urls.apply(content_type)
df['contains_image_by_ct'] = df.content_type.apply(lambda t: ('image/jpeg' or 'image/gif') in t) #TODO: Add more

###Identify Images by File Extension

In [ ]:
def extensions(urls):
    l = []
    for url in urls:
        if url[-4:-3] == ".":
            s = url[-3:].lower()
            if s.isalpha():
                l.append(s)
        elif url[-5:-4] == ".":
            s = url[-4:].lower()
            if s.isalpha():
                l.append(s)
    return l

df['extensions'] = df.expanded_urls.apply(extensions)
df['contains_image_by_ex'] = df.extensions.apply(lambda t: ('jpg' or 'jpeg' or 'gif' or 'png' or 'svg' or 'bmp') in t)

###Compare Results of Image Identification Mechanisms

In [ ]:
df['contains_image_by_ct'].value_counts()

In [ ]:
df['contains_image_by_ex'].value_counts()

In [ ]:
df['contains_image_by_e'].value_counts()

In [ ]:
df['contains_image_by_ee'].value_counts()

###TODO: Deal with Photo Sites like Instagram and Flickr

In [ ]:
photo_sites = ['instagram.com'] #TODO: Add more

###Compute Perceptual Hash

In [ ]:
hash1 = pHash.imagehash('/Users/cbopp/Desktop/testing_images/image3.jpg')
hash2 = pHash.imagehash('/Users/cbopp/Desktop/testing_images/image4.jpg')
print 'Hamming distance:',pHash.hamming_distance(hash1, hash2),'(',hash1,'/',hash2,')'

In [ ]:
print '%08x' % (hash1)
print '%08x' % (hash2)

In [ ]:
print '%d' % (pHash.hamming_distance(hash1, hash2))

In [ ]:
digest1 = pHash.image_digest('/Users/cbopp/Desktop/image3.jpg',1.0,1.0,180)
digest2 = pHash.image_digest('/Users/cbopp/Desktop/image4.jpg',1.0,1.0,180)
print 'Cross-correlation:',pHash.crosscorr(digest1,digest2)

###Examine Retweet Behavior

Reference http://0-proquestcombo.safaribooksonline.com.libraries.colorado.edu/book/web-development/twitter/9781449303624/1dot-the-recipes/id2867843?uicode=ucb

###Pickling

In [ ]:
#df.to_pickle('/Users/cbopp/Data/Nepal/nepal-earthquake-4-27-2248-head-1k.pkl'
#df.to_pickle('/Users/cbopp/Data/Nepal/nepal-earthquake-4-27-2248-head-25.pkl'
df.to_pickle('/Users/cbopp/Data/Sandy_Image/2012_Hurricane_Sandy-2k.pkl')

In [ ]:
df = pd.read_pickle('/Users/cbopp/Data/Sandy_Image/2012_Hurricane_Sandy-2k.pkl')

###Get Images

In [ ]:
image_directory = '/Users/cbopp/Data/Sandy_Image/images/'

def download_images(l):
    filenames = []
    
    for url_original in l:

        u_original = urlparse(url_original)
        
        if u_original.netloc == "p.twimg.com":
            url = "http://pbs.twimg.com/media" + u_original.path
        else:
            url = url_original
        
        u = urlparse(url)
        filename = (u.netloc + u.path).replace("/","_")

        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(image_directory + filename, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            filenames.append(filename)
        else:
            print("Connection Error: {}".format(url))
    
    return filenames

df['e_files'] = df['e_media'][df['contains_image_by_e'] == True].apply(download_images)
df['ee_files'] = df['ee_media'][df['contains_image_by_ee'] == True].apply(download_images)
df['ex_files'] = df['expanded_urls'][df['contains_image_by_ex'] == True].apply(download_images)
df['ct_files'] = df['expanded_urls'][df['contains_image_by_ct'] == True].apply(download_images)

In [ ]:
image_directory = '/Users/cbopp/Data/Sandy_Image/images/'

def download_images(row):
    
    f_df = pd.DataFrame
    
    for _, v in row['e_media'].iteritems():
    
    for url_original in l:

        u_original = urlparse(url_original)
        
        if u_original.netloc == "p.twimg.com":
            url = "http://pbs.twimg.com/media" + u_original.path
        else:
            url = url_original
        
        u = urlparse(url)
        filename = (u.netloc + u.path).replace("/","_")

        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(image_directory + filename, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            filenames.append(filename)
        else:
            print("Connection Error: {}".format(url))
    
    return filenames

filename_df = df[df['contains_image_by_e'] == True].apply(download_images, axis=1)

# I need a data frame like this as output from the download_images function,
# which can then be merged with the main df.
#
# |-------|-------------------------------|
# |       |  pbs.twimg.com_media_A6FV...  |
# |       |-------------------------------|
# |  76   |  pbs.twimg.com_media_A6Fr...  |
# |       |-------------------------------|
# |       |  pbs.twimg.com_media_A6Fq...  |
# |-------|-------------------------------|

###Output to Visualization

####Format to ImagePlot Macro for ImageJ:
Data file that references the image filename (in one or more directories)

####Data File Format
File Type: Tab Delimited Text File  
Cannot contain commas  
Line 1: Headers  
Line 2+: One line per image  

Headers:
* Must have at least 2 columns
* Optional - filename or full file path
* X & Y can only be integers or floating points
* Text columns can be labels


In [ ]:
#df['contains_image'] = (df['contains_image_by_e'] | df['contains_image_by_ee'] | df['contains_image_by_ex'] | df['contains_image_by_ct'])
df['contains_image'] = (df['contains_image_by_e'] | df['contains_image_by_ex'] | df['contains_image_by_ct'])

In [ ]:
df.keys()

In [ ]:
df.get_value(76,'e_files')

In [ ]:
def output_tsv(df_selection):
    
    df_selection.set_index(['e_files'], inplace=True)
    df_selection.index.name = None
    return df_selection
    
    #for r in df.iteritems():
    #    print(r)
    
    #for file_list in df['e_files']:
    #    for file in file_list:
    #        print(file)

output_tsv(df.ix[df['contains_image'] == True])

In [ ]:
#Output headers to tsv file.
#Output (for now) basic fields like user ID, tweet ID, and others that are easy to output and are int/float per filename.
#Each image filename will be one line even if this duplicates tweets (it will).